### 0. Import Libs

In [1]:
import pandas as pd 
import os 
import constant as c
import sys
sys.path.append('../')
from util.get_coord_vw import GeoCoderVworld

### 1. Get all the data regarding building

In [2]:
# We get all raw data for make data view
ssd_path = c.SSD_PATH
address_path = os.path.join(ssd_path, 'address_202304061732.csv')
title_path = os.path.join(ssd_path, 'title_202304061735.csv')
jijigu_path = os.path.join(ssd_path, 'jijigu_202304061738.csv')
building_path = os.path.join(ssd_path, 'building_202304061737.csv')
pnubjdcd_path = os.path.join(ssd_path, 'pnubjdcd.txt')
address = pd.read_csv(address_path, encoding = 'cp949')
title = pd.read_csv(title_path, encoding = 'cp949')
jijigu = pd.read_csv(jijigu_path, encoding = 'cp949')
building = pd.read_csv(building_path, encoding = 'cp949')
pnubjdcd = pd.read_csv(pnubjdcd_path, encoding = 'cp949', sep =  '\t')
pnubjdcd = pnubjdcd[pnubjdcd['법정동명'].str.contains('서울특별시')]
pnubjdcd = pnubjdcd[pnubjdcd['폐지여부'] == '존재']

df_bd = pd.merge(address, building[['pnu', 'bdnm']], on = 'pnu')
df_bd = pd.merge(df_bd, \
               title[['pnu', 'mainpurpscd', 'mainpurpscdnm', 'etcpurps', 'platarea', \
                'archarea', 'bcrat', 'totarea', 'vlrat', 'heit', 'grndflrcnt', 'ugrndflrcnt', \
                    'rideuseelvtcnt', 'emgenuseelvtcnt', 'useaprday']], on = 'pnu')
df_bd = pd.merge(df_bd, \
                jijigu[(jijigu['jijigugbcdnm'] == '용도지역코드')&(jijigu['repryn'] == 1)][['pnu', 'jijigucdnm']], on = 'pnu') # 용도지역코드만 따오고 대표여부가 참(=1)인 데이터만
df_bd = df_bd[df_bd['sigungunm'].isin(c.CBD_NMS)]
df_bd = df_bd[df_bd['heit'] > 10] # building taller than 10m
df_bd = df_bd.drop_duplicates(subset=['pnu'], keep='first')
df_bd = df_bd[df_bd['mainpurpscd'].isin(c.MAINPURPSCD_IN_INTEREST)]
df_bd = df_bd.sort_values(by=['pnu'], ascending=False)

gcw = GeoCoderVworld() # Class declaration for getting coordinates
xs = []
ys = []
for i in range(len(df_bd['newplatplc'])):
    addr = df_bd.iloc[i].newplatplc
    x, y = gcw.AddrToCoord(addr)
    xs.append(x)
    ys.append(y)

df_bd['x'] = xs 
df_bd['y'] = ys

/var/folders/ql/lx2yjbg126q9v8ykbp3cnfww0000gp/T/ipykernel_56333/2898056959.py:9: DtypeWarning: Columns (3,4,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  title = pd.read_csv(title_path, encoding = 'cp949')
/var/folders/ql/lx2yjbg126q9v8ykbp3cnfww0000gp/T/ipykernel_56333/2898056959.py:10: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  jijigu = pd.read_csv(jijigu_path, encoding = 'cp949')


### 2. Get individual Land Price

In [3]:
ilp_path = os.path.join(ssd_path, 'AL_11_D151_20230320.csv')
ilp = pd.read_csv(ilp_path, encoding = 'cp949')
ilp = ilp[ilp['기준연도'].isin([2019, 2020, 2021, 2022])]
ilp['법정동코드_str']= ilp['법정동코드'].astype(str)
ilp['구'] = ilp['법정동코드_str'].str.slice(0, 5)

# Get ilp data for four years
merged2019 = df_bd.merge(ilp[ilp['기준연도'] == 2019][['고유번호', '기준연도', '공시지가']], left_on='pnu', right_on='고유번호')
merged2019 = merged2019.rename(columns={'공시지가': 'ilp2019'})
merged2019 = merged2019.drop(['기준연도', '고유번호'], axis=1)

merged2020 = merged2019.merge(ilp[ilp['기준연도'] == 2020][['고유번호', '기준연도', '공시지가']], left_on='pnu', right_on='고유번호')
merged2020 = merged2020.rename(columns={'공시지가': 'ilp2020'})
merged2020 = merged2020.drop(['기준연도', '고유번호'], axis=1)

merged2021 = merged2020.merge(ilp[ilp['기준연도'] == 2021][['고유번호', '기준연도', '공시지가']], left_on='pnu', right_on='고유번호')
merged2021 = merged2021.rename(columns={'공시지가': 'ilp2021'})
merged2021 = merged2021.drop(['기준연도', '고유번호'], axis=1)

merged2022 = merged2021.merge(ilp[ilp['기준연도'] == 2022][['고유번호', '기준연도', '공시지가']], left_on='pnu', right_on='고유번호')
merged2022 = merged2022.rename(columns={'공시지가': 'ilp2022'})
merged2022 = merged2022.drop(['기준연도', '고유번호'], axis=1)

del merged2019, merged2020, merged2021
df_bd = merged2022.copy()
df_bd.to_csv("../asset/preprocess/features/df_bd.csv")
del merged2022

/var/folders/ql/lx2yjbg126q9v8ykbp3cnfww0000gp/T/ipykernel_56333/364585105.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  ilp = pd.read_csv(ilp_path, encoding = 'cp949')


In [4]:
# Read df_bd
df_bd = pd.read_csv("../asset/preprocess/features/df_bd.csv", index_col = 0)